# Differential Privacy

This is a notebook introducing the concepts behind differential privacy. Some parts of it are based on the wonderful book "Programming Differential Privacy" by Joseph P. Near and Chiké Abuah, available online: [https://programming-dp.com](https://programming-dp.com).

## Definition of Differential Privacy

In the following we will consider datasets about individuals, i.e., a table where each row corresponds to an individual.
Our goal is to perform some analytical task on some dataset $D$. We will express the analytical task as a function $F$ that is applied to $D$. and 

Formally, we consider $F$ to be a *randomized algorithm* that takes as input a dataset $D$ and produces an output $y$. Randomized means that the algorithm will not deterministically always output a single $y$ but rather introduces a distribution on the range of function $F$. This means that it is possible for $F$ to have many possible outputs for the same input $D$. Most often the range of $Y$ is continuous, and we will use $Y$ to denote a subset of its values.
Intuitively, we want to quantify how much information about individuals in $D$, the function $F$ "leaks".

A randomized algorithm $F$ is **$\epsilon$-differentially private** for $\epsilon \ge 0$, if for any two datasets $D$ and $D'$ that differ by one individual (row), and for any set $Y$ it holds that:
$$
\frac{\mathsf{Pr}[F(D) \in Y]}{\mathsf{Pr}[F(D') \in Y]} \leq e^\epsilon.
$$

Note that the role of $D$ and $D'$ is interchangeable. Therefore it holds that:
$$
\frac{\mathsf{Pr}[F(D') \in Y]}{\mathsf{Pr}[F(D) \in Y]} \leq e^\epsilon,
$$
and thus one may also write:
$$
\frac{1}{e^\epsilon} \leq \frac{\mathsf{Pr}[F(D) \in Y]}{\mathsf{Pr}[F(D') \in Y]} \leq e^\epsilon.
$$

Further note that if we set $\epsilon=0$, we get:
$$
\mathsf{Pr}[F(D) \in Y] = \mathsf{Pr}[F(D') \in Y], \quad \text{if } \ \epsilon=0.
$$

That is, the function $F$ reveals nothing about any individual in $D$ for $\epsilon=0$. This also means that the function retains no information from $D$ and thus is useless to the data analyst. Therefore, we typically want functions that are differentially private with $\epsilon >0$.


Clearly, the smaller the value of $\epsilon$ is, the higher the privacy is. So what values of $\epsilon$ should we care about? The general consensus is that $\epsilon$ should not be greater than 10.

## How to achieve Differential Privacy

Let's see how we can put differential privacy in practice.

We will use the ```Adult``` dataset, as was modified by the authors of the "Programming Differential Privacy" book to include identifying information. 

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True
adult = pd.read_csv("adult_with_pii.csv")

adult.head()

,Name,DOB,SSN,Zip,Workclass,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Hours per week,Country,Target,Age,Capital Gain,Capital Loss
0,Karrie Trusslove,9/7/1967,732-14-6110,64152,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,56,2174,0
1,Brandise Tripony,6/7/1988,150-19-2766,61523,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35,0,0
2,Brenn McNeely,8/6/1991,725-59-9860,95668,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,32,0,0
3,Dorry Poter,4/6/2009,659-57-4974,25503,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,14,0,0
4,Dick Honnan,9/16/1951,220-93-3811,75387,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,72,0,0


We will use the following analytical task, posed as a query:  

"*How many individuals in the dataset are 40 years old or older?*"

The answer is 17450.

In [2]:
f = adult[adult['Age'] >= 40].shape[0]
f 

17450

Let's now try to answer this question with some privacy guarantees. We will achieve differential privacy through the *Laplace mechanism*.

According to the Laplace mechanism, for a function $f(x)$ which returns a number, the following definition of $F(x)$ satisfies $\epsilon$-differential privacy:

$$
F(x) = f(x) + \textsf{Lap}\left(\frac{s}{\epsilon}\right),
$$

where $s$ is the *sensitivity* of $f$, and $\textsf{Lap}(S)$ denotes sampling from the Laplace distribution with center 0 and scale $S$.


The *sensitivity* of a function $f$ is the amount $f$'s output changes when its input changes by one individual. Here, we will only work with counting queries, which have a sensitivity of 1. If a query asks for the number of individuals with a particular property, adding or removing exactly one individual can only change the query's output by at most 1.


Let's now implement this randomized function $F$ for $\epsilon=0.1$, and see its output.

In [3]:
def laplace_mech(f, sensitivity, epsilon):
    return f + np.random.laplace(loc=0, scale=sensitivity/epsilon)


laplace_mech(f, 1, 0.1)

17452.875072918494

You can see the effect of the noise by running the above cell multiple times. Each time, the output changes, but most of the time, the answer is close enough to the true answer 17450.

It's better to visualize this with a histogram.

In [4]:
def laplace_hist(f, sensitivity, epsilon, n_iteration=1000, n_bins=50):
    plt.hist([laplace_mech(f, sensitivity, epsilon) for i in range(1000)], bins=50, label=r"$\epsilon="+str(epsilon)+"$")
    plt.axvline(x=f, color="red");
    plt.legend();

laplace_hist(f, 1, 0.1)

Error in callback <function _draw_all_if_interactive at 0x00000190DF593BA0> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: latex was not able to process the following string:
b'lp'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmpjww4fxdh 844f707e0917555faafa721cd5105dbd.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.25 (MiKTeX 24.1) (preloaded format=latex.fmt)
 restricted \write18 enabled.
entering extended mode
(844f707e0917555faafa721cd5105dbd.tex
LaTeX2e <2023-11-01> patch level 1
L3 programming layer <2024-01-04>
(C:\Users\micha\AppData\Local\Programs\MiKTeX\tex/latex/base\article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(C:\Users\micha\AppData\Local\Programs\MiKTeX\tex/latex/base\size10.clo))

! LaTeX Error: File `type1cm.sty' not found.

Type X to quit or <RETURN> to proceed,
or enter new name. (Default extension: sty)

Enter file name: 
! Emergency stop.
<read *> 
         
l.7 \usepackage
               {type1ec}^^M
No pages of output.
Transcript writt

Error in callback <function flush_figures at 0x00000190DF591300> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

Let's change the epsilon value to see what happens. First let's set $\epsilon=1$, and then $\epsilon=5$.

In [ ]:
laplace_hist(f, 1, 1)

In [ ]:
laplace_hist(f, 1, 5)

See how the variance around 17450 decreases, as $\epsilon$ increases.

Now let's consider a malicious count query that tries to learn if a particular individual, Karrie Trusslove, has income less than or equal to 50K.

We will pose the following  query:  

"*How many individuals are named Karrie Trusslove and have income less than or equal to 50K?*"

The answer is 1.

In [4]:
f = adult[(adult['Name'] == 'Karrie Trusslove') & (adult['Target'] == '<=50K')].shape[0]
f

1

Let's see the answer after applying the Laplace mechanism with $\epsilon=1$.

In [5]:
laplace_mech(f, 1, 1)

4.19271410960952

Is the true answer 0 or 1? There's too much noise to be able to reliably tell. This is how differential privacy is *intended* to work; it adds enough noise that the results of a malicious query will be useless to the adversary.

# Local Differential Privacy

So far we have only considered the *central model* of differential privacy, in which the sensitive data is collected centrally in a single dataset. In this setting, we assume that the *analyst* is malicious, but that there is a *trusted data curator* who holds the dataset and correctly executes the differentially private mechanisms the analyst specifies.

That setting is often not realistic. In many cases, the data curator and the analyst are *the same*, and no trusted third party actually exists to hold the data and execute mechanisms. In fact, the organizations which collect the most sensitive data tend to be exactly the ones we *don't* trust; such organizations certainly can't function as trusted data curators.

An alternative to the central model of differential privacy is the *local model of differential privacy*, in which data is made differentially private before it leaves the control of the data subject. For example, you might add noise to your data *on your device* before sending it to the data curator. In the local model, the data curator does not need to be trusted, since the data they collect *already* satisfies differential privacy.

The local model thus has one huge advantage over the central model: data subjects don't need to trust anyone else but themselves. This advantage has made it popular in real-world deployments, including the ones by [Google](https://github.com/google/rappor) and [Apple](https://www.apple.com/privacy/docs/Differential_Privacy_Overview.pdf).

Unfortunately, the local model also has a significant drawback: the accuracy of query results in the local model is typically *orders of magnitude lower* for the same privacy cost as the same query under central differential privacy. This huge loss in accuracy means that only a small handful of query types are suitable for local differential privacy, and even for these, a large number of participants is required.

## Definition of Local Differential Privacy

The goal of local differential privacy (LDP) is to protect a sensitive value.

Let $X$ denote a set of input values, and $Y$ a set of output values. Given an input value $x \in X$, LDP will output instead a value $y \in Y$. Note that $Y$ can coincide with $X$, in which case LDP will essentially "hide" $x$ among the other values in $X$.

A randomized algorithm $F$ is **$\epsilon$-locally differentially private** for $\epsilon \ge 0$, if for any two distinct values $x \in X$ and $x' \in X$, and for any value $y \in Y$ that $F$ can output it holds that:
$$
\frac{\mathsf{Pr}[F(x) = y]}{\mathsf{Pr}[F(x') =y]} \leq e^\epsilon.
$$

This implies that looking at the output $y$ of an LDP algorithm, one cannot deduce with certainty what the original sensitive value $x$ is.

## Randomized Response

[Randomized response](https://en.wikipedia.org/wiki/Randomized_response) is a mechanism for local differential privacy which was first proposed in a 1965 [paper by S. L. Warner](https://www.jstor.org/stable/2283137?seq=1#metadata_info_tab_contents). At the time, the technique was intended to improve bias in survey responses about sensitive issues, and it was not originally proposed as a mechanism for differential privacy (which wouldn't be invented for another 40 years). After differential privacy was developed, statisticians realized that this existing technique *already* satisfied the definition.

The goal is for a subject to respond to a sensitive "yes" or "no" question.

The randomized response protocol is the following:
- Flip a coin.
- If the coin is heads, answer the question truthfully.
- Else, flip another coin.
    - If the second coin is heads, answer "yes".
    - Else, answer "no".

The randomization in this algorithm comes from the two coin flips. As in all other differentially private algorithms, this randomization creates uncertainty about the true answer, which is the source of privacy.

Let's see why randomized response satisfies LDP. We will encode a "yes" answer as a 1, and a "no" answer as a 0. The actual response will be the input value $x$, and the randomized response will be the output value $y$.

First, let's see what the output is given the input and the outcome of the coin tosses.


| input | 1st flip | 2nd flip | probability | output |
|---|---|---|---|---|
| 0 | heads |  |  1/2 | 0 |
|  | tails | heads |  1/4 | 1 |
|  |  | tails |  1/4 | 0 |
| 1 | heads |  |  1/2 | 1 |
|  | tails | heads |  1/4 | 1 |
|  |  | tails |  1/4 | 0 |


So if the input value is $x=0$, the output value is $y=0$ with probability $3/4$. Conversely, if $x=1$, the output values is $y=1$ with probability $3/4$.

The following table summarizes the "randomness" in the randomized response protocol. It gives the conditional probability of $P(y|x)$.

| input x | output y=0| output y=1 |
|---|---|---|
| 0 |  3/4 |  1/4 |
| 1 |  1/4 |  3/4 |

Let's now apply the definition of LDP to see what $\epsilon$ guarantee we get.
First, consider $y=0$:

$$ 
\frac{\mathsf{Pr}[F(x=0) = 0]}{\mathsf{Pr}[F(x=1) =0]} = \frac{3/4}{1/4} = 3.
$$

Similarly for $y=1$, we get:

$$ 
\frac{\mathsf{Pr}[F(x=1) = 1]}{\mathsf{Pr}[F(x=0) =1]} = \frac{3/4}{1/4} = 3.
$$

The probability ratio is always 3.

Therefore, **randomized response satisfies LDP** with $\epsilon = \ln(3) \approx 1.099$.


### Generalized randomized response

We can generalize randomized response by explicitly setting the "randomness". Observe that the randomness we saw, defines the following row-stochastic matrix:

$$
F = 
\begin{bmatrix}
3/4 & 1/4 \\
1/4 & 3/4 
\end{bmatrix}.
$$

We can arbitrarily set the values of that matrix by introducing two parameters:
- $p$, the probability that a 1 ("yes" answer) will be preserved, i.e., randomized to a 1, and
- $q$, the probability that a 0 ("no" answer) will be preserved, i.e., randomized to a 0.

It's safe to assume that $p$ and $q$ are greater than $1/2$. This defines the following matrix:

$$
F = 
\begin{bmatrix}
q & 1-q \\
1-p & p 
\end{bmatrix} 
$$

**Note:** The simple randomized response with the two fair coin tosses, corresponds to $p=q=3/4$.

#### Question
What is the $\epsilon$ value that this generalized randomized response satisfies?

#### Answer 

We can compute the probability ratios for the two possible outputs $y=0$ and $y=1$, which gives us the following ratios, $\frac{q}{1-p}$ and $\frac{p}{1-q}$, which are both greater than 1.

Therefore the privacy guarantee comes from the larger of these two ratios:
$$
\epsilon = \ln( \max\{ \frac{q}{1-p}, \frac{p}{1-q} \}  ).
$$

Setting the values of $p$, and $q$ we can effectively control the privacy guarantee. For example, when we set $p=0.7$, and $q=0.6$, we get $\epsilon = \ln(0.6/0.3) \approx 0.693 $.

#### Question
Assume that $p=q$. What is the probability $p$ required to achieve a given privacy guarantee $\epsilon$?

#### Answer

Setting $p=q$, allows us to *reverse engineer* the value of $p$ that provides a given level of privacy protection $\epsilon$.

Specifically, to satisfy $\epsilon$-LDP, we can set:

$$
p = q = \frac{e^\epsilon}{1 + e^\epsilon}.
$$


For example, to achieve 2-LDP, we can set $p=q \approx 0.881$.

#### Python Implementation

Here's a Python implementation of generalized randomized response.

In [6]:
import random

def rand_resp(x, p=0.75, q=0.75):
    toss = random.random()
    if x == 0:
        y = 0 if toss <= q else 1
    else:
        y = 1 if toss <= p else 0
    return y

And here are some functions to derive $\epsilon$ from $p$ and $q$, and to derive $p=q$ for a target $\epsilon$. 

In [7]:
import math

def get_epsilon(p=0.75, q=0.75):
    return math.log( max(q/(1-p), p/(1-q)) )

print(f"For plain RR, epsilon is {get_epsilon():.3f}")

def get_p_q(epsilon):
    p = math.exp(epsilon)/(1+math.exp(epsilon))
    return p, p

epsilon = 2
print(f"To guarantee {epsilon}-LDP set both p and q to {get_p_q(epsilon)[0]:.3f}")



For plain RR, epsilon is 1.099
To guarantee 2-LDP set both p and q to 0.881


Let's use that to verify that randomized response outputs $y=x$ three times more often than not.

In [ ]:
n_outputs = 10000
y_0 = [rand_resp(0) for i in range(n_outputs)]
# y_1 = [rand_resp(1) for i in range(n_outputs)]

print(f"There are {n_outputs - sum(y_0)} outputs y=0 and {sum(y_0)} outputs y=1 out of {n_outputs} inputs of x=1")

plt.hist(y_0, bins=50, label=r"$y | x=0$");
plt.legend();

There are 7524 outputs y=0 and 2476 outputs y=1 out of 10000 inputs of x=1


## The Utility of Randomized Response

Let's investigate what is the effect of randomized response on count queries. We're going to pose the following query:

"*How many individuals in the dataset have occupation 'Sales'?*"

To answer this let's extract a Boolean array that has 1 whenever the column `adult['Occupation']` takes the value `'Sales'`.

In [ ]:
truth = adult['Occupation'].apply(lambda x: 1 if x == 'Sales' else 0).values

n_salespeople = np.sum(truth)
n_people = len(truth)

print(f"There are {n_salespeople} salespeople among {n_people} people.")

Let's now apply randomized response to the question "Are you a salesperson?".

In [ ]:
responses = np.array([rand_resp(x) for x in truth])
n_rep_salespeople = np.sum(responses)

print(f"Applying the randomized response protocol, there are {n_rep_salespeople} reported as salespeople.")

There are 3650 salespersons in the dataset, but when we count their randomized responses, we estimate there are many more! This is a *very very bad* estimate.

Let's understand what's going on and see how we can get a more accurate estimate.

Assume that the proportion of salespersons in the dataset is $r$. Looking at the randomness matrix for randomized response, we can determine the following expected proportions of people with randomized response $y$ given their actual response $x$.


|  |  | # people with rr $y=0$ | # people with rr $y=1$ |
|---|---|---|---|
| # people with $x=0$ | $1-r$ |  $\frac{3}{4}(1-r)$ |  $\frac{1}{4}(1-r)$ |
| # people with $x=1$ | $r$ |  $\frac{1}{4}r$ |  $\frac{3}{4}r$ |

Therefore, we will observe the output $y=1$ from $\frac{1}{4}$ of the people that have $x=0$, and from $\frac{3}{4}$ of the people that have $x=1$. In total, since there is a proportion of $r$ people with $x=1$, we expect to observe a proportion of $\frac{1}{2}r + \frac{1}{4}$ among the global population that give a randomized response of $y=1$.

Therefore, given a reported count of $n_1^{rep}$ for $y=1$ out of $n$ people, we can solve the equation $n_1^{rep} = (\frac{1}{2}r + \frac{1}{4})n$ for $r$, and obtain

$$
r = 2\frac{n_1^{rep}}{n} - \frac{1}{2}.
$$

Therefore, we can estimate the number of salespeople as follows.

In [ ]:
n_est_salespeople = 2*n_rep_salespeople - 0.5*n_people

print(f"The estimated number of salespeople is {n_est_salespeople}.")
print(f"This is quite close to the actual number of {n_salespeople} salespeople.")

#### Question
Repeat the analysis for the generalized RR with arbitrary $p$ and $q$.

#### Answer

First, we will derive the expected number of people who report $y$ given their actual value $x$.

|  |  | # people with rr $y=0$ | # people with rr $y=1$ |
|---|---|---|---|
| # people with $x=0$ | $1-r$ |  $q(1-r)$ |  $(1-q)(1-r)$ |
| # people with $x=1$ | $r$ |  $(1-p)r$ |  $pr$ |

Therefore, we expect to observe $y=1$ from a total of $n_1^{rep} = ((p+q-1)r + 1-q)n$ people. 

Solving for $r$, we can estimate $r$ as follows.

$$
r =  \frac{1}{p+q-1} (\frac{n_1^{rep}}{n} + q - 1 ).
$$

#### Python Implementation

Let's first define the functions to apply RR on a vector `truth` of values. Then, define the function that estimates the number of people using the estimation of $r$.

In [ ]:
def apply_rand_resp(truth, p=0.75, q=0.75):
    return np.array([rand_resp(x, p, q) for x in truth])

def estimate(responses, p=0.75, q=0.75):
    n_people = len(responses)
    n_reported = np.sum(responses)
    return (n_reported/n_people + q - 1)/(p+q-1)*n_people

    

Here's an example with randomized response of arbitrary $p$, $q$ values. Observe that setting $p$, $q$ higher than their default values, increases accuracy, at the expense of privacy loss ($\epsilon$ increases).

In [ ]:
p, q = 0.95, 0.85
epsilon = get_epsilon(p, q)
print(f"We will apply {epsilon:.3f}-LDP setting p={p}, q={q}.")

responses = apply_rand_resp(truth, p, q)

n_est_salespeople = estimate(responses, p, q)

print(f"There is an estimated {n_est_salespeople:.0f} salespeople.")
print(f"This is very close to the actual number of {n_salespeople} salespeople.")